In [ ]:
!pip install datasets
!pip install orjson
!pip install git+https://github.com/openai/CLIP.git
!pip install faiss-gpu
!pip install accelerate
!pip install -U bitsandbytes
!pip install nltk
!pip install sentence-transformers
!pip install qwen_vl_utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-td36416a
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLI

In [ ]:
from datasets import load_dataset
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import transformers
import orjson
import torch
from transformers import CLIPProcessor, CLIPModel
import clip
import faiss
import numpy as np
import json
import copy
from transformers import Blip2Processor, Blip2ForConditionalGeneration, BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from nltk.translate.bleu_score import sentence_bleu
from sentence_transformers import SentenceTransformer
from sklearn.metrics import f1_score
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.translate.meteor_score import meteor_score
from peft import LoraConfig, get_peft_model


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
#Upload data/vqa_rad/VQA_RAD Dataset Public.json from the Github repository to your Google Drive and put the absolute path to it below
VQA_RAD_ANNOTS_FILE = r""

#Upload data/mimic_cxr/clip_text_features.json from the Github repository to your Google Drive and put the absolute path to it below
CLIP_TEXT_FEATURES_FILE = r""

#Upload data/mimic_cxr/clip-imp-pretrained_128_6_after_4.pt from the Github repository to your Google Drive and put the absolute path to it below
CLIP_MODEL_FILE = r""

#Upload data/vqa_rad/images from the Github repository to your Google Drive and put the absolute path to it below
IMAGE_PATH_PREFIX = r""

#Upload data/vqa_rad/vqa_annotations_promptmrg.json from the Github repository to your Google Drive and put the absolute path to it below
CHEX_ANNOTS_FILE = r""

TOP_K = 21

#Extract only the Chest X-ray [image, question, answer] triplets from the entire VQA-RAD dataset
def get_chex_image_data(vqa_rad_annotations_file):
    #Annotations originally stored in JSON format
    with open(vqa_rad_annotations_file, "rb") as f:
        annotations = orjson.loads(f.read())

    chex_data = [] #[[image_path, question, answer]]
    for entry in annotations:
      if entry["image_organ"] == "CHEST":
        chex_data.append([entry["image_name"], entry["question"], entry["answer"]])

    return chex_data


#Store all CLIP text features in Vector Database
def get_report_db_clip_features():
    #Load large JSON file with all clip features 270,790 x (1 x 512)
    with open(CLIP_TEXT_FEATURES_FILE, 'rb') as f:
        clip_text_features = orjson.loads(f.read())

    #Create FAISS Vector Database with all the CLIP text features stored in it
    embedding_dimension = len(clip_text_features[0])
    faiss_index = faiss.IndexFlatL2(embedding_dimension)
    faiss_index.add(np.array(clip_text_features).astype("float32"))
    faiss.write_index(faiss_index, "faiss_index.faiss")

    return clip_text_features, faiss_index


#Extract indices of Top-K most similar text features from Vector Database
def get_clip_indices(image_paths, clip_text_features, faiss_index):
    #Use ViT-B/32 because that is same backbone used in the CLIP model
    clip_model, clip_processor = clip.load("ViT-B/32", device="cuda", jit=False)
    clip_model.load_state_dict(torch.load(CLIP_MODEL_FILE, map_location="cuda"))
    clip_model.eval()

    all_top_matches = []
    for image_path in image_paths:
      image = Image.open(IMAGE_PATH_PREFIX + image_path)
      model_input = clip_processor(image).unsqueeze(0).to("cuda")

      #Pass image features through CLIP to get embeddings
      with torch.no_grad():
          image_features = clip_model.encode_image(model_input)

      #Normalize the CLIP output embeddings and move to CPU device for direct comparison with Vector Database
      image_features /= image_features.norm(dim=-1, keepdim=True) #Size is 1x512 and matches each embedding in clip_text_features length = 512
      image_features = image_features.cpu().numpy().astype('float32')

      #Do Similarity Search to find Top-K most similar vectors in Vector Database and return their indices
      distance_matrix, top_matches = faiss_index.search(image_features, k=TOP_K)
      all_top_matches.append(top_matches)

    return all_top_matches


#Create JSON file with annotations to be downloaded and sent to PromptMRG for inferencing to generate the reports for the images in the VQA RAD testing dataset
def create_json_file(all_top_matches, image_names):
  data = {"test": []}

  for image_name, clip_indices in zip(image_names, all_top_matches):
    clip_indices = clip_indices[0].tolist()

    #Create JSON entry that follows the format PromptMRG expects each image to be annotated in
    entry = {
        "id": image_name,
        "report": "NA",
        "image_path": [
            image_name
        ],
        "split": "test",
        "view": [
            "PA"
        ],
        "labels": 14 * [0],
        "clip_indices": clip_indices
    }
    data["test"].append(entry)

  with open(CHEX_ANNOTS_FILE, 'w') as chex_annots_json:
    json.dump(data, chex_annots_json, indent=4)

  return data



if __name__ == "__main__":
    #First process dataset to only include all 794 CHEX images
    image_data = get_chex_image_data(VQA_RAD_ANNOTS_FILE)
    image_names = list(set([image_data[i][0] for i in range(len(image_data))]))
    print(f"{len(image_names)} CHEX Images and {len(image_data)} total QA Pairs")

    #Then extract and store all clip_text_features in FAISS Vector Store
    clip_text_features, faiss_index = get_report_db_clip_features()

    #Then get clip_indices for all 794 CHEX images extracted earlier
    all_top_matches = get_clip_indices(image_names, clip_text_features, faiss_index)
    print(f"Acquired {len(all_top_matches)} sets for {len(image_names)} Images of Top {TOP_K} matches")

    #Now create new JSON vqa_rad_annotations_promptmrg.json file to be passed as input to PromptMRG to generate reports for these images
    json_file_data = create_json_file(all_top_matches, image_names)
    print(f"Created CHEX Annotations JSON File and Stored in Google Drive ready for local download")

107 CHEX Images and 794 total QA Pairs


100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 146MiB/s]
<ipython-input-3-ff8e076285b0>:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  clip_model.

Acquired 107 sets for 107 Images of Top 21 matches
Created CHEX Annotations JSON File and Stored in Google Drive ready for local download


In [ ]:
# At this point, here is what has happened
# 1. We have taken the full VQA-RAD dataset and extracted only the [image, question, answer] triplets for Chest X-rays

# 2. For each of these images we have found the Top-K most similar feature embeddings from the Vector Database that stores feature embeddings from the MIMIC-CXR dataset the CLIP model was trained on

# 3. Given these [image, question, answer] and Top-K most similar feature embedding information we have created a new annotation JSON file that stores all of this information together

# 4. We are still missing one thing - and that is the medical reports for all these Chest-Xray images. We do not have these reports already because VQA-RAD is a VQA dataset not a medical report generation one
#    Therefore, we have to use PromptMRG to generate these reports for these images.

# 5. Now, the annotation JSON file simply needs to be inputted into PromptMRG so that it can generate reports for these images

# 6. This requires going back to your local Python environment and running inferencing for PromptMRG using "./test_vqa_rad.sh"

# 7. However, to save time, we have already done this step and have saved the medical reports that PromptMRG generated for these images

# 8. You can find them in the GitHub repository at "results/promptmrg/experiment_results/base_iu_model/test/base_iu_model_vqa_rad_mrg_test_log.json"

# 9. All you have to do is upload that file into your Google Drive and put the absolute path to it below in the next cell

# 10. Then simply continue running this notebook!

In [ ]:
#Upload results/promptmrg/experiment_results/base_iu_model/test/base_iu_model_vqa_rad_mrg_test_log.json from the Github repository to your Google Drive and put the absolute path to it below
IMAGE_REPORTS_FILE = r""

#The goal here is to take a list storing [image, question, answer] triplets and convert it to -> [image, question, answer, report] by adding in the reports that PromptMRG just created
with open(IMAGE_REPORTS_FILE, "rb") as f:
    image_report_dict = orjson.loads(f.read())

image_data = get_chex_image_data(VQA_RAD_ANNOTS_FILE)

#Print out some sample [image, question, answer] entries
print(image_data[0])
print(image_data[56])


#Append report to the triplet
image_data_with_report = copy.deepcopy(image_data)

for i in range(len(image_data_with_report)):
  image_name = image_data_with_report[i][0]
  report = image_report_dict[image_name]
  image_data_with_report[i].append(report)

#Print the same sample values now with the report appended
print(image_data_with_report[0])
print(image_data_with_report[56])

['synpic29265.jpg', 'Are the lungs normal appearing?', 'No']
['synpic33422.jpg', 'Has the midline of the mediastinum shifted?', 'No']
['synpic29265.jpg', 'Are the lungs normal appearing?', 'No', 'the heart is normal in size. the mediastinum is unremarkable. the lungs are clear. there is no pleural effusion or pneumothorax. the xxxx are intact.']
['synpic33422.jpg', 'Has the midline of the mediastinum shifted?', 'No', 'the heart is normal in size. the mediastinum is unremarkable. the lungs are clear. there is no pleural effusion or pneumothorax. the skeletal structures are normal.']


In [ ]:
#Now load the Moondream 2 VLM from Huggingface to perform the experiments
model_id = "vikhyatk/moondream2"
revision = "2024-08-26"
model = AutoModelForCausalLM.from_pretrained(
    model_id, trust_remote_code=True, revision=revision
)
tokenizer = AutoTokenizer.from_pretrained(model_id, revision=revision)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/319 [00:00<?, ?B/s]

configuration_moondream.py:   0%|          | 0.00/3.34k [00:00<?, ?B/s]

moondream.py:   0%|          | 0.00/7.20k [00:00<?, ?B/s]

modeling_phi.py:   0%|          | 0.00/63.1k [00:00<?, ?B/s]

region_model.py:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

fourier_features.py:   0%|          | 0.00/558 [00:00<?, ?B/s]

vision_encoder.py:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.74G [00:00<?, ?B/s]

In [ ]:
#Perform Experiment 1: VLM Not Fine Tuned and Knowledge-Enhanced MVQA With Reports

results_with_reports = {}
for i, image_q_pair in enumerate(image_data_with_report):
  #Iterate through first 160 images = images in testing dataset
  if i <= 160:
    image = Image.open(IMAGE_PATH_PREFIX + image_q_pair[0])
    enc_image = model.encode_image(image)
    question = image_q_pair[1]
    text_prompt = f"Question: {question}\nMedical Report as reference: {image_q_pair[3]}"
    vlm_output = model.answer_question(enc_image, text_prompt, tokenizer)
    results_with_reports[i] = [text_prompt, image_q_pair[2], vlm_output]

    print(f"Image: {image_q_pair[0]}")
    print(f"{text_prompt}")
    print(f"GT Answer: {image_q_pair[2]}")
    print(f"VLM Answer: {vlm_output}")


Image: synpic29265.jpg
Question: Are the lungs normal appearing?
Medical Report as reference: the heart is normal in size. the mediastinum is unremarkable. the lungs are clear. there is no pleural effusion or pneumothorax. the xxxx are intact.
GT Answer: No
VLM Answer: Yes
Image: synpic29265.jpg
Question: Is there evidence of a pneumothorax
Medical Report as reference: the heart is normal in size. the mediastinum is unremarkable. the lungs are clear. there is no pleural effusion or pneumothorax. the xxxx are intact.
GT Answer: No
VLM Answer: No
Image: synpic28602.jpg
Question: What type of imaging does this not represent?
Medical Report as reference: the heart is normal in size. the mediastinum is unremarkable. the lungs are clear. there is no pleural effusion or pneumothorax. the xxxx are intact.
GT Answer: ultrasound
VLM Answer: Radiographic
Image: synpic29265.jpg
Question: Is this a MRI of the chest?
Medical Report as reference: the heart is normal in size. the mediastinum is unrema

In [ ]:
#Perform analysis of Experiment 1
#Calculate SBERT Similarity Score, METEOR Score, and F1 Score


sbert_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
total_sim_score = 0

binary_answer_ground_truth = []
binary_answer_vlm = []
binary_answers = ["yes", "no"]

total_meteor_score = 0

for entry in results_with_reports.keys():
  ground_truth = results_with_reports[entry][1]
  vlm_prediction = results_with_reports[entry][2]

  #Check if question has binary answer (ie a close ended question) and if so, keep track of ground truth and VLM answers
  if ground_truth.lower() in binary_answers and vlm_prediction.lower() in binary_answers:
    binary_answer_ground_truth.append(ground_truth.lower())
    binary_answer_vlm.append(vlm_prediction.lower())

  #If answer is incorrect, count similarity as zero manually becasue Sentence Bert does not always think yes is orthogonal to no
  if ground_truth.lower() == "yes" and vlm_prediction.lower() == "no" or ground_truth.lower() == "no" and vlm_prediction.lower() == "yes":
    sim_score = 0

  else:
    #SBERT Score
    ground_truth_embedding = sbert_model.encode(ground_truth)
    vlm_prediction_embedding = sbert_model.encode(vlm_prediction)
    sim_score = sbert_model.similarity(ground_truth_embedding, vlm_prediction_embedding).item() #returns Tensor so call .item() to get scalar value

    #METEOR Score
    meteor_score_with = meteor_score([nltk.word_tokenize(ground_truth.lower())], nltk.word_tokenize(vlm_prediction.lower()))
    total_meteor_score += meteor_score_with

  total_sim_score += sim_score

#Display Final Results for Experiment 1
avg_sim_score_with = round(total_sim_score / len(results_with_reports), 2)
print(f"Average Similarity Score With Reports: {avg_sim_score_with}")

avg_meteor_score_with = round(total_meteor_score / len(results_with_reports), 2)
print(f"Average METEOR Score With Reports: {avg_meteor_score_with}")

binary_f1_score = round(f1_score(binary_answer_ground_truth, binary_answer_vlm, average="binary", pos_label="yes"), 2)
print(f"Binary F1 Score: {binary_f1_score}")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Average Similarity Score With Reports: 0.52
Average METEOR Score With Reports: 0.22
Binary F1 Score: 0.64


In [ ]:
#Perform Experiment 2: VLM Not Fine Tuned and Independent MVQA Without Reports

results_without_reports = {}
for i, image_q_pair in enumerate(image_data):
  #Iterate through first 160 images = images in testing dataset
  if i <= 160:
    image = Image.open(IMAGE_PATH_PREFIX + image_q_pair[0])
    enc_image = model.encode_image(image)
    question = image_q_pair[1]
    text_prompt = f"Question: {question}"
    vlm_output = model.answer_question(enc_image, text_prompt, tokenizer)
    results_without_reports[i] = [text_prompt, image_q_pair[2], vlm_output]

    print(f"Image: {image_q_pair[0]}")
    print(f"Question: {image_q_pair[1]}")
    print(f"GT Answer: {image_q_pair[2]}")
    print(f"VLM Answer: {vlm_output}")

Image: synpic29265.jpg
Question: Are the lungs normal appearing?
GT Answer: No
VLM Answer: No, the lungs appear to be damaged or affected, as they are depicted in a black and white radiographic image.
Image: synpic29265.jpg
Question: Is there evidence of a pneumothorax
GT Answer: No
VLM Answer: Yes, there is evidence of a pneumothorax in the image. The chest radiograph shows a large amount of air in the space between the lungs, which is a characteristic of a pneumothorax. This condition can cause shortness of breath, chest pain, and sometimes a feeling of suffocation. The radiograph helps the medical professional to identify and diagnose the presence of a pneumothorax, allowing for appropriate treatment and care for the patient.
Image: synpic28602.jpg
Question: What type of imaging does this not represent?
GT Answer: ultrasound
VLM Answer: This image represents an X-ray, which is a type of medical imaging that uses electromagnetic radiation to create an image of the body's internal str

In [ ]:
#Perform analysis of Experiment 2
#Calculate SBERT Similarity Score, METEOR Score, and F1 Score

sbert_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
total_sim_score = 0

binary_answer_ground_truth = []
binary_answer_vlm = []
binary_answers = ["yes", "no"]

total_meteor_score = 0

for entry in results_without_reports.keys():
  ground_truth = results_without_reports[entry][1]
  vlm_prediction = results_without_reports[entry][2]

  #Check if question has binary answer (ie a close ended question) and if so, keep track of ground truth and VLM answers
  index = vlm_prediction.find(",")
  if index != -1 and ground_truth.lower() in binary_answers and vlm_prediction[:index].lower() in binary_answers:
    binary_answer_ground_truth.append(ground_truth.lower())
    binary_answer_vlm.append(vlm_prediction[:index].lower())

  #If answer is incorrect, count similarity as zero manually becasue Sentence Bert does not always think yes is orthogonal to no
  if ground_truth.lower() == "yes" and vlm_prediction.lower() == "no" or ground_truth.lower() == "no" and vlm_prediction.lower() == "yes":
    sim_score = 0

  else:
    #SBERT Score
    ground_truth_embedding = sbert_model.encode(ground_truth)
    vlm_prediction_embedding = sbert_model.encode(vlm_prediction)
    sim_score = sbert_model.similarity(ground_truth_embedding, vlm_prediction_embedding).item() #returns Tensor so call .item()

    #METEOR Score
    meteor_score_with = meteor_score([nltk.word_tokenize(ground_truth.lower())], nltk.word_tokenize(vlm_prediction.lower()))
    total_meteor_score += meteor_score_with

  total_sim_score += sim_score

#Display Final Results for Experiment 2
avg_sim_score_with = round(total_sim_score / len(results_with_reports), 2)
print(f"Average Similarity Score With Reports: {avg_sim_score_with}")

avg_meteor_score_with = round(total_meteor_score / len(results_with_reports), 2)
print(f"Average METEOR Score With Reports: {avg_meteor_score_with}")

binary_f1_score = round(f1_score(binary_answer_ground_truth, binary_answer_vlm, average="binary", pos_label="yes"), 2)
print(f"Binary F1 Score: {binary_f1_score}")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Average Similarity Score With Reports: 0.22
Average METEOR Score With Reports: 0.1
Binary F1 Score: 0.61


In [ ]:
#Run this if you want to push the dataset to huggingface hub to visualize it on the Web UI
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
#Create formal train and test dataset objects to finetune the VLM

#ACKNOWLEDGEMENT OF CREDIT: https://medium.com/google-developer-experts/ml-story-fine-tune-vision-language-model-on-custom-dataset-8e5f5dace7b1
  #Utilized this tutorial for guidance on how to convert a dictionary to dataset object and push it to Huggingface Hub for visualization

from datasets import Dataset, Image

#Enter your HF ID if you want to push the datasets to your HF account for visualization
HF_ID = r""

#Reverse the image data due to the choice of images used for inferencing in Experiments 1 and 2
dataset_dict = {"ids": [], "image_names": [], "images": [], "questions": [], "answers": []}
image_data_copy = copy.deepcopy(image_data)
image_data_reversed = image_data_copy[::-1]

for i, data_triplet in enumerate(image_data_reversed):
  dataset_dict["ids"].append(str(i))
  dataset_dict["image_names"].append(str(data_triplet[0]))
  dataset_dict["images"].append(str(IMAGE_PATH_PREFIX + data_triplet[0]))
  dataset_dict["questions"].append(str(data_triplet[1]))
  dataset_dict["answers"].append(str(data_triplet[2]))

dataset = Dataset.from_dict(dataset_dict)
#Convert all the images to type Image()
dataset = dataset.cast_column("images", Image())

split_dataset = dataset.train_test_split(test_size=0.2, shuffle=False)

#If you want to push to HF Hub, then run the following lines (my default)
# split_dataset.push_to_hub(HF_ID)
# train_dataset = load_dataset(HF_ID, split="train")
# eval_dataset = load_dataset(HF_ID, split="test")

#If you dont want to push to HF Hub then run the following lines
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

[['synpic29265.jpg', 'Are the lungs normal appearing?', 'No'], ['synpic29265.jpg', 'Is there evidence of a pneumothorax', 'No'], ['synpic28602.jpg', 'What type of imaging does this not represent?', 'ultrasound'], ['synpic29265.jpg', 'Is this a MRI of the chest?', 'no'], ['synpic28602.jpg', 'What is not pictured in this image?', 'The extremities'], ['synpic28602.jpg', 'Is the trachea midline?', 'yes'], ['synpic28602.jpg', 'Is there evidence of an aortic aneurysm?', 'No'], ['synpic28602.jpg', 'Is there blunting of the costovertebral angles?', 'No'], ['synpic42202.jpg', 'Is there evidence of an aortic aneurysm?', 'yes'], ['synpic42202.jpg', 'Which organ system is abnormal in this image?', 'Cardiovascular'], ['synpic29265.jpg', 'Is there airspace consolidation on the left side?', 'Yes'], ['synpic42202.jpg', 'Is there a pneumothorax?', 'no'], ['synpic29265.jpg', 'Is there a fracture?', 'no'], ['synpic29265.jpg', 'How is the patient oriented?', 'Posterior-Anterior'], ['synpic28602.jpg', 'Is 

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/635 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/159 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

No files have been modified since last commit. Skipping to prevent empty commit.


In [ ]:
#Use this to verify the datasets have been created correctly
print(train_dataset[0])
print(train_dataset[1])
print(train_dataset[2])

print(eval_dataset[-1])
print(eval_dataset[-2])
print(eval_dataset[-3])

{'ids': '0', 'image_names': 'synpic26764.jpg', 'images': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x1192 at 0x7F53C4345FC0>, 'questions': 'Is this film taken in a PA modality?', 'answers': 'Yes'}
{'ids': '1', 'image_names': 'synpic26764.jpg', 'images': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x1192 at 0x7F53C4346650>, 'questions': 'Is this a PA film?', 'answers': 'Yes'}
{'ids': '2', 'image_names': 'synpic38531.jpg', 'images': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x873 at 0x7F53C43470D0>, 'questions': 'Is the apical aeration normal or decreased?', 'answers': 'Normal'}
{'ids': '793', 'image_names': 'synpic29265.jpg', 'images': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=480x503 at 0x7F53C4347130>, 'questions': 'Are the lungs normal appearing?', 'answers': 'No'}
{'ids': '792', 'image_names': 'synpic29265.jpg', 'images': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=480x503 at 0x7F53C4347160>, 'questions': 'Is ther

In [ ]:
#Configure the hyperparameters
EPOCHS = 2
BATCH_SIZE = 2
GRAD_ACCUM_STEPS = 1
LR = 1e-5
USE_WANDB = False

In [ ]:
#Finetune the VLM for MVQA on the VQA-RAD training dataset

#ACKNOWLEDGEMENT OF CREDIT: https://github.com/vikhyat/moondream/blob/main/notebooks/Finetuning.ipynb
  #The following code was provided as the standard code for finetuning the Moondream 2 VLM. Given its unique architecture, the developers
  #released sample code as a tutorial for how to finetune their model. The only modifications I make to their code involve changing
  #some of the for loops and iteration structure to load my VQA-RAD dataset correctly. As the focus of this project is not exploring different
  #finetuning techniques, I relied on their default finetuning code. Future work for this project would definitely involve digging into this code
  #even more deeply and optimizing it further for the AIMRG and MVQA tasks.


from torch.utils.data import DataLoader
from bitsandbytes.optim import Adam8bit
import math
from einops import rearrange
from tqdm import tqdm

ANSWER_EOS = "<|endoftext|>"

# Number of tokens used to represent each image.
IMG_TOKENS = 729

DEVICE = "cuda"

model.to(DEVICE)

def collate_fn(batch):
    images = [sample['images'] for sample in batch]
    images = [model.vision_encoder.preprocess(image) for image in images]

    labels_acc = []
    tokens_acc = []

    for sample in batch:
        # print(sample)
        toks = [tokenizer.bos_token_id]
        labs = [-100] * (IMG_TOKENS + 1)

        # for qa in sample['qa']:
        q_t = tokenizer(
            f"\n\nQuestion: {sample['questions']}\n\nAnswer:",
            add_special_tokens=False
        ).input_ids
        toks.extend(q_t)
        labs.extend([-100] * len(q_t))

        a_t = tokenizer(
            f" {sample['answers']}{ANSWER_EOS}",
            add_special_tokens=False
        ).input_ids
        toks.extend(a_t)
        labs.extend(a_t)

        tokens_acc.append(toks)
        labels_acc.append(labs)

    max_len = -1
    for labels in labels_acc:
        max_len = max(max_len, len(labels))

    attn_mask_acc = []

    for i in range(len(batch)):
        len_i = len(labels_acc[i])
        pad_i = max_len - len_i

        labels_acc[i].extend([-100] * pad_i)
        tokens_acc[i].extend([tokenizer.eos_token_id] * pad_i)
        attn_mask_acc.append([1] * len_i + [0] * pad_i)

    return (
        images,
        torch.stack([torch.tensor(t, dtype=torch.long).to(DEVICE) for t in tokens_acc]),
        torch.stack([torch.tensor(l, dtype=torch.long).to(DEVICE) for l in labels_acc]),
        torch.stack([torch.tensor(a, dtype=torch.bool).to(DEVICE) for a in attn_mask_acc]),
    )

def compute_loss(batch):
    images, tokens, labels, attn_mask = batch

    images = [image.to(DEVICE) for image in images]
    tokens = tokens.to(DEVICE)
    labels = labels.to(DEVICE)
    attn_mask = attn_mask.to(DEVICE)

    with torch.no_grad():
        img_embs = model.vision_encoder(images)

    tok_embs = model.text_model.get_input_embeddings()(tokens)
    inputs_embeds = torch.cat((tok_embs[:, 0:1, :], img_embs, tok_embs[:, 1:, :]), dim=1)

    outputs = model.text_model(
        inputs_embeds=inputs_embeds,
        labels=labels,
        attention_mask=attn_mask,
    )

    return outputs.loss

def lr_schedule(step, max_steps):
    x = step / max_steps
    if x < 0.1:
        return 0.1 * LR + 0.9 * LR * x / 0.1
    else:
        return 0.1 * LR + 0.9 * LR * (1 + math.cos(math.pi * (x - 0.1))) / 2

dataloaders = {
    "train": DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        collate_fn=collate_fn,
    )
}

model.text_model.train()
model.text_model.transformer.gradient_checkpointing_enable()

total_steps = EPOCHS * len(dataloaders["train"]) // GRAD_ACCUM_STEPS
optimizer = Adam8bit(
    [
        {"params": model.text_model.parameters()},
    ],
    lr=LR * 0.1,
    betas=(0.9, 0.95),
    eps=1e-6
)

if USE_WANDB:
    import wandb
    wandb.init(
        project="moondream-ft",
        config={
            "EPOCHS": EPOCHS,
            "BATCH_SIZE": BATCH_SIZE,
            "GRAD_ACCUM_STEPS": GRAD_ACCUM_STEPS,
            "LR": LR,
        }
    )

i = 0
for epoch in range(EPOCHS):
    for batch in tqdm(dataloaders["train"], desc=f"Epoch {epoch + 1}/{EPOCHS}"):
        i += 1

        loss = compute_loss(batch)
        loss.backward()
        print(f"\nLoss: {round(loss.item(), 3)} | Epoch: {epoch}/{EPOCHS} | {i}/{len(dataloaders['train'])}")

        if i % GRAD_ACCUM_STEPS == 0:
            optimizer.step()
            optimizer.zero_grad()

            lr = lr_schedule(i / GRAD_ACCUM_STEPS, total_steps)
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr

        if USE_WANDB:
            wandb.log({
                "loss/train": loss.item(),
                "lr": optimizer.param_groups[0]['lr']
            })

if USE_WANDB:
    wandb.finish()

Epoch 1/2:   0%|          | 1/318 [00:01<09:25,  1.78s/it]


Loss: 7.144 | Epoch: 0/2 | 1/318


Epoch 1/2:   1%|          | 2/318 [00:03<08:38,  1.64s/it]


Loss: 2.962 | Epoch: 0/2 | 2/318


Epoch 1/2:   1%|          | 3/318 [00:04<08:27,  1.61s/it]

...


Loss: 0.791 | Epoch: 1/2 | 634/318


Epoch 2/2: 100%|█████████▉| 317/318 [08:01<00:01,  1.52s/it]


Loss: 0.208 | Epoch: 1/2 | 635/318


Epoch 2/2: 100%|██████████| 318/318 [08:02<00:00,  1.52s/it]


Loss: 1.043 | Epoch: 1/2 | 636/318


In [ ]:
#Save the safetensors file if you wish
SAVE_PATH = r""
model.save_pretrained(SAVE_PATH)

In [ ]:
#Perform Experiment 3: VLM Fine Tuned and Independent MVQA Without Reports

model.eval()

ft_results_without_reports = {}

for i, entry in enumerate(eval_dataset):
  print(f"Question: {entry['questions']}")
  print(f"Ground Truth: {entry['answers']}")
  vlm_output = model.answer_question(model.encode_image(entry['images']), entry['questions'], tokenizer=tokenizer)
  print(f"Moondream: {vlm_output}")
  ft_results_without_reports[i] = [entry["questions"], entry["answers"], vlm_output]

Question: Where are the most infiltrates located?
Ground Truth: Left lung
Moondream: Right lung
Question: Which lung has more extensive infiltration?
Ground Truth: Left
Moondream: Right
Question: Are the borders of the heart clearly visualized?
Ground Truth: yes
Moondream: No
Question: Is this image a CT scan?
Ground Truth: no
Moondream: no
Question: Which cardiac border is more prominently visualized?
Ground Truth: Left
Moondream: Left
Question: Which side of the cardiac border is more prominent?
Ground Truth: Left
Moondream: Left
Question: Where is the cardiac border more obscured?
Ground Truth: Right
Moondream: Right side
Question: On which side is the cardiac border more obscured?
Ground Truth: Right
Moondream: Right
Question: What rib is the lesion located inferior to?
Ground Truth: 7th rib
Moondream: Bilateral
Question: At which rib is the lesion located?
Ground Truth: 7th rib
Moondream: 3rd rib
Question: What is the most important abnormality found in this image?
Ground Truth: W

In [ ]:
#Perform analysis of Experiment 3
#Calculate SBERT Similarity Score, METEOR Score, and F1 Score

sbert_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
total_sim_score = 0

binary_answer_ground_truth = []
binary_answer_vlm = []
binary_answers = ["yes", "no"]

total_meteor_score = 0

for entry in ft_results_without_reports.keys():
  ground_truth = ft_results_without_reports[entry][1]
  vlm_prediction = ft_results_without_reports[entry][2]

  #Check if question has binary answer (ie a close ended question) and if so, keep track of ground truth and VLM answers
  if ground_truth.lower() in binary_answers and vlm_prediction.lower() in binary_answers:
    binary_answer_ground_truth.append(ground_truth.lower())
    binary_answer_vlm.append(vlm_prediction.lower())

  #If answer is incorrect, count similarity as zero manually becasue Sentence Bert does not always think yes is orthogonal to no
  if ground_truth.lower() == "yes" and vlm_prediction.lower() == "no" or ground_truth.lower() == "no" and vlm_prediction.lower() == "yes":
    sim_score = 0

  else:
    #SBERT Score
    ground_truth_embedding = sbert_model.encode(ground_truth)
    vlm_prediction_embedding = sbert_model.encode(vlm_prediction)
    sim_score = sbert_model.similarity(ground_truth_embedding, vlm_prediction_embedding).item() #returns Tensor so call .item()

    #METEOR Score
    meteor_score_with = meteor_score([nltk.word_tokenize(ground_truth.lower())], nltk.word_tokenize(vlm_prediction.lower()))
    total_meteor_score += meteor_score_with

  total_sim_score += sim_score

#Display Final Results for Experiment 3
avg_sim_score_with = round(total_sim_score / len(ft_results_without_reports), 2)
print(f"Average Similarity Score With FT without Reports: {avg_sim_score_with}")

avg_meteor_score_with = round(total_meteor_score / len(ft_results_without_reports), 2)
print(f"Average METEOR Score With FT without Reports: {avg_meteor_score_with}")

binary_f1_score = round(f1_score(binary_answer_ground_truth, binary_answer_vlm, average="binary", pos_label="yes"), 2)
print(f"Binary F1 Score: {binary_f1_score}")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Average Similarity Score With FT without Reports: 0.64
Average METEOR Score With FT without Reports: 0.29
Binary F1 Score: 0.64


In [ ]:
#Perform Experiment 4: VLM Fine Tuned and Knowledge-Enhanced MVQA With Reports

model.eval()

ft_results_with_reports = {}

for i, entry in enumerate(eval_dataset):
  question = entry["questions"]

  #Get report for current image
  image_path = entry["image_names"]
  for image_data in image_data_with_report:
    if image_data[0] == image_path:
      report = image_data[3]
      break

  text_prompt = f"{question}\nMedical Report as reference: {report}"
  vlm_output = model.answer_question(model.encode_image(entry['images']), text_prompt, tokenizer=tokenizer)

  ft_results_with_reports[i] = [entry["questions"], entry["answers"], vlm_output]

  print(f"Question: {text_prompt}")
  print(f"Ground Truth: {entry['answers']}")
  print(f"Moondream: {vlm_output}\n")

Question: Where are the most infiltrates located?
Medical Report as reference: the heart is normal in size. the mediastinum is unremarkable. there is no pleural effusion or pneumothorax. there is mild degenerative changes of the spine.
Ground Truth: Left lung
Moondream: Right side

Question: Which lung has more extensive infiltration?
Medical Report as reference: the heart is normal in size. the mediastinum is unremarkable. there is no pleural effusion or pneumothorax. there is mild degenerative changes of the spine.
Ground Truth: Left
Moondream: Right lung

Question: Are the borders of the heart clearly visualized?
Medical Report as reference: heart size within normal limits stable mediastinal and hilar contours. no focal alveolar consolidation no definite pleural effusion seen. no typical findings of pulmonary edema. no typical findings of pulmonary edema.
Ground Truth: yes
Moondream: yes

Question: Is this image a CT scan?
Medical Report as reference: the lungs are clear. there is n

In [ ]:
#Perform analysis of Experiment 4
#Calculate SBERT Similarity Score, METEOR Score, and F1 Score

sbert_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
total_sim_score = 0

binary_answer_ground_truth = []
binary_answer_vlm = []
binary_answers = ["yes", "no"]

total_meteor_score = 0

for entry in ft_results_with_reports.keys():
  ground_truth = ft_results_with_reports[entry][1]
  vlm_prediction = ft_results_with_reports[entry][2]

  #Check if question has binary answer (ie a close ended question) and if so, keep track of ground truth and VLM answers
  if ground_truth.lower() in binary_answers and vlm_prediction.lower() in binary_answers:
    binary_answer_ground_truth.append(ground_truth.lower())
    binary_answer_vlm.append(vlm_prediction.lower())

  #If answer is incorrect, count similarity as zero manually becasue Sentence Bert does not always think yes is orthogonal to no
  if ground_truth.lower() == "yes" and vlm_prediction.lower() == "no" or ground_truth.lower() == "no" and vlm_prediction.lower() == "yes":
    sim_score = 0

  else:
    #SBERT Score
    ground_truth_embedding = sbert_model.encode(ground_truth)
    vlm_prediction_embedding = sbert_model.encode(vlm_prediction)
    sim_score = sbert_model.similarity(ground_truth_embedding, vlm_prediction_embedding).item() #returns Tensor so call .item()

    #METEOR Score
    meteor_score_with = meteor_score([nltk.word_tokenize(ground_truth.lower())], nltk.word_tokenize(vlm_prediction.lower()))
    total_meteor_score += meteor_score_with

  total_sim_score += sim_score

#Display Final Results for Experiment 4
avg_sim_score_with = round(total_sim_score / len(ft_results_without_reports), 2)
print(f"Average Similarity Score With FT with Reports: {avg_sim_score_with}")

avg_meteor_score_with = round(total_meteor_score / len(ft_results_without_reports), 2)
print(f"Average METEOR Score With FT with Reports: {avg_meteor_score_with}")

binary_f1_score = round(f1_score(binary_answer_ground_truth, binary_answer_vlm, average="binary", pos_label="yes"), 2)
print(f"Binary F1 Score: {binary_f1_score}")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Average Similarity Score With FT with Reports: 0.61
Average METEOR Score With FT with Reports: 0.28
Binary F1 Score: 0.75


In [ ]:
torch.cuda.empty_cache()

In [1]:
!pip freeze > Stage2B_MVQA_VLM.txt